In [123]:
import pandas as pd
import numpy as np
import ast
import requests
from sklearn.preprocessing import OneHotEncoder

In [124]:
api_key_auth3 = 'fad9ac13c7b36b3e05f6b63be16e74f0'

# Get genre ids

In [125]:
genre_dictionary = {}

response = requests.get(f"https://api.themoviedb.org/3/genre/movie/list?api_key={api_key_auth3}&language=en-US")
response = response.json()

for genre in response['genres']:
    genre_dictionary[genre['id']] = genre['name']

# Load in dataset and process

In [126]:
feature_df = pd.read_csv(r'./MovieDataEnhanced.csv', encoding = "ISO-8859-1")
feature_df['box_office'] = feature_df['domestic_box_office'] + feature_df['international_box_office']
feature_df.head()

,id,genre_ids,original_language,adult,popularity,vote_average,vote_count,movie_name,production_year,production_budget,domestic_box_office,international_box_office,genre,running_time,box_office
0,16781,"[18, 35, 10749]",en,False,9.536,6.374,179,Madea's+Family+Reunion,2006,10000000,63257940,62581,Comedy,NaN,63320521
1,32740,"[28, 878]",hi,False,9.654,6.378,193,Krrish,2006,10000000,1430721,31000000,Action,NaN,32430721
2,9526,"[18, 35, 10402]",en,False,10.004,6.300,261,A+Prairie+Home+Companion,2006,10000000,20342852,6373339,Comedy,105.0,26716191
3,214,"[27, 53, 80]",en,False,36.265,6.382,3825,Saw+III,2006,10000000,80238724,83638091,Horror,NaN,163876815
4,9794,"[35, 10749]",en,False,12.064,5.537,531,Employee+of+the+Month,2006,10000000,28444855,9920000,Comedy,108.0,38364855


In [127]:
feature_df.describe()

,id,popularity,vote_average,vote_count,production_year,production_budget,domestic_box_office,international_box_office,running_time,box_office
count,1.407000e+03,1407.000000,1407.000000,1407.000000,1407.000000,1.407000e+03,1.407000e+03,1.407000e+03,1308.000000,1.407000e+03
mean,1.224441e+05,26.298989,6.288824,3151.733475,2010.823028,5.295481e+07,6.341492e+07,9.326907e+07,109.764526,1.566840e+08
std,1.622140e+05,28.205945,1.220506,4153.456730,3.522023,5.214777e+07,7.970790e+07,1.502839e+08,20.046836,2.215133e+08
min,3.500000e+01,0.600000,0.000000,0.000000,2006.000000,1.000000e+07,0.000000e+00,0.000000e+00,0.000000,9.069000e+03
25%,1.138100e+04,11.026500,5.872500,526.500000,2008.000000,1.985000e+07,1.134943e+07,9.226397e+06,97.000000,2.723338e+07
50%,4.123300e+04,17.835000,6.402000,1588.000000,2010.000000,3.500000e+07,3.738405e+07,3.840000e+07,108.000000,7.623500e+07
75%,2.184135e+05,31.047500,6.951000,4008.500000,2014.000000,6.500000e+07,8.165488e+07,1.018845e+08,121.000000,1.810320e+08
max,1.110807e+06,298.358000,10.000000,33502.000000,2018.000000,4.250000e+08,7.605076e+08,2.015838e+09,201.000000,2.776345e+09


## Group by box office revenue

In [128]:
# Group by box_office
# Groups numerical revenue, currently 6 bins
num = [0,1,2,3,4,5]

revenue_in = pd.qcut(feature_df['box_office'], q=6, labels=num)

enc = OneHotEncoder().fit(np.array(revenue_in).reshape(-1,1))
df = pd.DataFrame(enc.transform(np.array(revenue_in).reshape(-1,1)).toarray())
df['combine'] = df.values.tolist()
feature_df['box_office_group'] = df['combine']

## Convert genres list to one hot array

In [129]:
feature_df['genre_ids'] = [ast.literal_eval(x) for x in feature_df['genre_ids']]
feature_df['genres'] = [[genre_dictionary[v] for v in g_array] for g_array in feature_df['genre_ids']]

In [130]:
feature_df.head()

,id,genre_ids,original_language,adult,popularity,vote_average,vote_count,movie_name,production_year,production_budget,domestic_box_office,international_box_office,genre,running_time,box_office,box_office_group,genres
0,16781,"[18, 35, 10749]",en,False,9.536,6.374,179,Madea's+Family+Reunion,2006,10000000,63257940,62581,Comedy,NaN,63320521,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]","[Drama, Comedy, Romance]"
1,32740,"[28, 878]",hi,False,9.654,6.378,193,Krrish,2006,10000000,1430721,31000000,Action,NaN,32430721,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[Action, Science Fiction]"
2,9526,"[18, 35, 10402]",en,False,10.004,6.300,261,A+Prairie+Home+Companion,2006,10000000,20342852,6373339,Comedy,105.0,26716191,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[Drama, Comedy, Music]"
3,214,"[27, 53, 80]",en,False,36.265,6.382,3825,Saw+III,2006,10000000,80238724,83638091,Horror,NaN,163876815,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[Horror, Thriller, Crime]"
4,9794,"[35, 10749]",en,False,12.064,5.537,531,Employee+of+the+Month,2006,10000000,28444855,9920000,Comedy,108.0,38364855,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]","[Comedy, Romance]"


In [131]:
exploded_df = feature_df[['id', 'genres']].explode(column='genres')
exploded_df = pd.get_dummies(exploded_df, columns=['genres']).groupby('id', as_index=False).sum()

In [132]:
select_features = ['popularity', 'vote_average'
                   , 'vote_count', 'production_budget', 'running_time']

## Get final dataset with select features

In [140]:
merged_features = pd.merge(left=feature_df[['id', 'movie_name', 'box_office_group'] + select_features]
                        , right=exploded_df
                        , left_on='id'
                        , right_on='id')

In [141]:
merged_features.describe()

,id,popularity,vote_average,vote_count,production_budget,running_time,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,...,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_Romance,genres_Science Fiction,genres_TV Movie,genres_Thriller,genres_War,genres_Western
count,1.407000e+03,1407.000000,1407.000000,1407.000000,1.407000e+03,1308.000000,1407.000000,1407.000000,1407.000000,1407.000000,...,1407.000000,1407.000000,1407.000000,1407.000000,1407.000000,1407.000000,1407.000000,1407.000000,1407.000000,1407.000000
mean,1.224441e+05,26.298989,6.288824,3151.733475,5.295481e+07,109.764526,0.276475,0.221748,0.102345,0.364606,...,0.069652,0.078181,0.041222,0.074627,0.151386,0.107321,0.005686,0.246624,0.035537,0.015636
std,1.622140e+05,28.205945,1.220506,4153.456730,5.214777e+07,20.046836,0.453728,0.425715,0.312452,0.484436,...,0.254650,0.268551,0.198875,0.262882,0.358552,0.314191,0.075217,0.437747,0.185198,0.135083
min,3.500000e+01,0.600000,0.000000,0.000000,1.000000e+07,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.138100e+04,11.026500,5.872500,526.500000,1.985000e+07,97.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.123300e+04,17.835000,6.402000,1588.000000,3.500000e+07,108.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.184135e+05,31.047500,6.951000,4008.500000,6.500000e+07,121.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.110807e+06,298.358000,10.000000,33502.000000,4.250000e+08,201.000000,2.000000,2.000000,2.000000,2.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,2.000000


In [142]:
#normalize
merged_features[select_features]=(merged_features[select_features]-merged_features[select_features].min())/(merged_features[select_features].max()-merged_features[select_features].min())

In [143]:
merged_features.head(3)

,id,movie_name,box_office_group,popularity,vote_average,vote_count,production_budget,running_time,genres_Action,genres_Adventure,...,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_Romance,genres_Science Fiction,genres_TV Movie,genres_Thriller,genres_War,genres_Western
0,16781,Madea's+Family+Reunion,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",0.030011,0.6374,0.005343,0.0,NaN,0,0,...,0,0,0,0,1,0,0,0,0,0
1,32740,Krrish,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",0.030407,0.6378,0.005761,0.0,NaN,1,0,...,0,0,0,0,0,1,0,0,0,0
2,9526,A+Prairie+Home+Companion,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",0.031583,0.6300,0.007791,0.0,0.522388,0,0,...,0,0,1,0,0,0,0,0,0,0


In [144]:
merged_features.describe()

,id,popularity,vote_average,vote_count,production_budget,running_time,genres_Action,genres_Adventure,genres_Animation,genres_Comedy,...,genres_History,genres_Horror,genres_Music,genres_Mystery,genres_Romance,genres_Science Fiction,genres_TV Movie,genres_Thriller,genres_War,genres_Western
count,1.407000e+03,1407.000000,1407.000000,1407.000000,1407.000000,1308.000000,1407.000000,1407.000000,1407.000000,1407.000000,...,1407.000000,1407.000000,1407.000000,1407.000000,1407.000000,1407.000000,1407.000000,1407.000000,1407.000000,1407.000000
mean,1.224441e+05,0.086308,0.628882,0.094076,0.103506,0.546092,0.276475,0.221748,0.102345,0.364606,...,0.069652,0.078181,0.041222,0.074627,0.151386,0.107321,0.005686,0.246624,0.035537,0.015636
std,1.622140e+05,0.094728,0.122051,0.123976,0.125657,0.099736,0.453728,0.425715,0.312452,0.484436,...,0.254650,0.268551,0.198875,0.262882,0.358552,0.314191,0.075217,0.437747,0.185198,0.135083
min,3.500000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.138100e+04,0.035017,0.587250,0.015715,0.023735,0.482587,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.123300e+04,0.057883,0.640200,0.047400,0.060241,0.537313,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.184135e+05,0.102256,0.695100,0.119650,0.132530,0.601990,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.110807e+06,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,2.000000


In [145]:
merged_features.to_csv('./cleaned_features.csv', index=False)